# Python Webmapping with Folium

### For this exercise we will use the following libraries: 
 

    >>> import pandas as pd
    >>> import geopandas as gpd
    >>> import folium
    >>> import branca
    >>> import os
    
Pandas/GeoPandas for reading geospatial data, Folium to generate the web map, Branca to create a colormap, and OS for system navigation as needed.

#### Now, create a map as m with folium.Map():
    >>> m = folium.Map()
    >>> m

#### There's your basic map. Now let's set the starting location to the Boulder area and change the basemap tiles:
    >>> m = folium.Map(location = [40.1, -105.4], tiles = 'Stamen Terrain', zoom_start = 10)
    >>> m

#### Now let's add a point to the map:

    >>> m = folium.Map(location = [40.1, -105.4], tiles = 'Stamen Terrain', zoom_start = 10)
    
    >>> folium.Marker([40.014984, -105.270546]).add_to(m)
    
    >>> m

In [ ]:
m = folium.Map(location = [40.1, -105.4], tiles = 'Stamen Terrain', zoom_start=10)

# insert code

m

#### There are multiple methods for adding and styling simple points on maps:

Try:

    >>> folium.Marker([40.014984, -105.270546], icon=folium.Icon(color='red')).add_to(m)
    
Or:

    >>> folium.vector_layers.CircleMarker(  
                                location = [40.014984, -105.270546],   
                                radius=10 ,  
                                weight=3,  
                                color='purple',  
                                fill_color='red',  
                                fill_opacity=1).add_to(m)


In [ ]:
m = folium.Map(location = [40.1, -105.4], tiles = 'Stamen Terrain', zoom_start=10)

# insert code

m

#### Now, let's make a choropleth map.

##### First, we will take a look at the Boulder County Census Tracts geojson layer.

    >>> m = folium.Map(location = [40.1, -105.4], tiles = 'CartoDB Positron', zoom_start = 10)
    >>> folium.GeoJson('Boulder_Co_Tracts.geojson').add_to(m)
    >>> m

##### Next, use Pandas to import the Boulder County Median Household Income csv file:

    >>> MHHI = pd.read_csv('Boulder_Co_MHHI.csv', dtype={'GEOID10':str})
    >>> MHHI

##### Now, we will use folium.Choropleth to join the Median Household Income tracts dataframe to the Boulder County Census Tracts geojson layer on the fly:

    >>> m = folium.Map(location = [40.1, -105.4], tiles = 'CartoDB Positron', zoom_start = 10)
    
    >>> folium.Choropleth(geo_data = 'Boulder_Co_Tracts.geojson',
                        data = MHHI,
                        columns = ['GEOID10', 'MHHI2014'],
                        key_on = 'feature.properties.GEOID10').add_to(m)
                        
    >>> m

In [ ]:
m = folium.Map(location = [40.1, -105.4], tiles = 'Stamen Terrain', zoom_start = 10)

# insert code

m

#### Next, we'll modify the color palette

    >>> m = folium.Map(location = [40.1, -105.4], tiles = 'Stamen Terrain', zoom_start = 10)
    
    >>> folium.Choropleth(geo_data = 'Boulder_Co_Tracts.geojson',
                        data = MHHI,
                        columns = ['GEOID10', 'MHHI2014'],
                        key_on = 'feature.properties.GEOID10',
                        fill_color = 'RdYlGn').add_to(m)
                        
    >>> m

*Note: fill_color accepts [Color Brewer palettes](https://colorbrewer2.org/#type=diverging&scheme=RdYlGn&n=5)*

In [ ]:
m = folium.Map(location = [40.1, -105.4], tiles = 'Stamen Terrain', zoom_start = 10)

folium.Choropleth(geo_data = r'C:\users\phwh9568\Workshops\Geopandas\Boulder_Co_Tracts.geojson',
                data = MHHI,
                columns = ['GEOID10', 'MHHI2014'],
                key_on = 'feature.properties.GEOID10',
                
                # insert code
                  
                 ).add_to(m)

m

##### Export your map:

    >>> m.save('my_map.html')

- ##### folium.choropleth is a quick & easy method of making a choropleth by doing an on the fly table join. 
- ##### If you want more funcitonality or have a geojson layer with the data already baked in, you will need to do a bit more work... 

### Advanced Choropleth 

##### Now we will make a choropleth map of 2020 presidential election results in Colorado. 

Start by importing the Colorado_Vote.geojson layer using GeoPandas:

    >>> CO_Vote = gpd.read_file('Colorado_Vote.geojson', driver='GeoJSON')
    
*Note:* This data is from the [New York Times](https://github.com/TheUpshot/presidential-precinct-map-2020).

##### View some details about the CO_Vote geodataframe:

    >>> CO_Vote
                
*Note: In this data, pct_dem_lead represents an index the victory percentage of either party. Negative values represent the percentage by which Republicans won and positive values represent the percent by which Democrats won.*

##### Initialize a map including this GeoJson layer:

    >>> m = folium.Map(location = [39, -105.7], tiles = 'CartoDB Positron', zoom_start=7)
    
    >>> folium.GeoJson(CO_Vote).add_to(m)
    
    >>> m

##### We will create a colormap based on the 'pct_dem_lead' field in the geojson:

    >>> colormap = branca.colormap.LinearColormap(
            colors=["#c93135", "#eaa9a9", "#92bde0", "#1375b7"],
            index=CO_Vote['pct_dem_lead'].quantile([0, .25, .5, .75]),
            vmin=-100,
            vmax=100)
            
    >>> colormap

##### Next we will create a style function:

    >>> def style(x):
        return {
            "fillColor": colormap(x["properties"]["pct_dem_lead"]),
            "color": "black",
            "weight": 0.25,
            "fillOpacity": .6}

##### Now we will style our voter disctrict polygons by the 'pct_dem_lead' field using our custom colormap:

    >>> m = folium.Map(location=[40.1, -105.7], tiles='CartoDB Positron', zoom_start=7)

    >>> folium.GeoJson(CO_Vote,style_function=style).add_to(m)

    >>> m

In [ ]:
m = folium.Map(location = [39, -105.7], tiles = 'CartoDB Positron', zoom_start=7)

# insert code

m

##### Now, we will finish our map by adding a tool tip so users can see the results data when they scroll over a polygon:

    >>> m = folium.Map(location=[40.1, -105.4], tiles='CartoDB Positron', zoom_start=7)

    >>> votes = folium.GeoJson(CO_Vote,
                              style_function=style,
                              tooltip=folium.GeoJsonTooltip(
                                      fields=['GEOID','votes_dem', 'votes_rep'], 
                                      aliases=['District ID', 'Democratic', 'Republican'])
                              ).add_to(m)
    >>> m

In [ ]:
m = folium.Map(location=[39, -105.7], tiles='CartoDB Positron', zoom_start=7)

votes = folium.GeoJson(CO_Vote,
                       style_function=style,
    
                       # insert code
                       
                      ).add_to(m)

m

##### Export your map:

    >>> m.save('co_vote_map.html')

## Nice Job!